In [ ]:
from frechet_fda.data_generation_helpers import (
    gen_params_scenario_one,
    make_truncnorm_pdf,
)
from frechet_fda.distribution_tools import (
    get_optimal_range,
    make_distribution_objects,
)
from frechet_fda.fda_funcs import (
    compute_centered_data,
    compute_cov_function,
    compute_fpc_scores,
    compute_principal_components,
)

In [ ]:
# Set parameters
n = 200
grid_size = 10000
trunc = 3
mus, sigmas = gen_params_scenario_one(n)
# Sort sigmas, because when summing Distribution instances something goes wrong
sigmas.sort()

In [ ]:
# Generate pdfs within truncation points
pdfs = make_truncnorm_pdf(-trunc, trunc, mus, sigmas, grid_size=grid_size)
# Make Distribution class objects
my_pdfs = make_distribution_objects(pdfs)
my_cdfs = [pdf.integrate() for pdf in my_pdfs]
my_qfs = [cdf.invert() for cdf in my_cdfs]
my_qdfs = [qf.differentiate() for qf in my_qfs]
# For numerical correction: shorten the range for smaller sigmas to get rid of
# numerical artifacts when computing integrals, derivatives and means later
new_ranges = get_optimal_range(my_pdfs)
# Generate pdfs again, this time within individual ranges
pdfs2 = [
    make_truncnorm_pdf(
        new_ranges[i][0], new_ranges[i][1], mus[i], sigmas[i], grid_size=grid_size,
    )[0]
    for i in range(n)
]

In [ ]:
# Generate all the distribution objects
new_pdfs = make_distribution_objects(pdfs2)
new_cdfs = [pdf.integrate() for pdf in new_pdfs]
new_qfs = [cdf.invert() for cdf in new_cdfs]
new_qdfs = [qf.differentiate() for qf in new_qfs]

In [ ]:
# Compute centered data, just to see whether it works
# One can clearly see how inappropriate it is to apply fda methods on densities
centered_pdfs = compute_centered_data(my_pdfs)
centered_pdfs[sigmas.argmax()].compare(my_pdfs[sigmas.argmax()])

In [ ]:
covariance_function = compute_cov_function(centered_pdfs)

In [ ]:
eigenvalues, eigenfunctions = compute_principal_components(
    my_pdfs[0].x, covariance_function,
)

In [ ]:
compute_fpc_scores(my_pdfs[0].x, centered_pdfs, eigenfunctions)